## PRACTICE

In [2]:
pip install findspark

Note: you may need to restart the kernel to use updated packages.


In [3]:
import findspark
findspark.init()

In [4]:
import pyspark
sc = pyspark.SparkContext(appName = "practice")

In [5]:
sc

<SparkContext master=local[*] appName=practice>

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [7]:
spark = SparkSession.builder.getOrCreate()

Consider daily product revenue data 
Product - product_id, pname, ptype, price

Customer - cust_id, cname, mobileno, city

orders – order_id, order_date, order_customer_id, order_status

order_items –  order_item_order_id, order_item_product_id, order_item_quantity, 
order_item_sub- total


Get details of all customers fro
- m-  pune city
Get details of orders with subtotal > ___ -
- in month __
Print orders in ascending order -
- of subtotal 
Print customer details with min order and max - order amount

Get orders which are either COM-
- PLETE or CLOSED
Get orders which are either COMPLETE or CLOSED and placed in mont-
- h of 2013 August
Get order items where order_item_subtotal is not equal to product of order_item_quantity and order_i-
- tem_product_price
Get all the orders which are placed on fi-
- rst of every month
Get count by-
-  status from orders
Get revenue for each order-
-  id from order items
Get daily product revenue (order_date and order_item_product_id are part of keys, order_item_subtotal is used for aggregation)



In [8]:
# Product - product_id, pname, ptype, price

product = [(1,"cups","kitchen items",50),
          (2,"pencil","art",20),
          (3,"paints","art",70),
          (4,"spoons","kitchen items",25),
          (5,"pillows","bedroom items",100),
          (6,"mirror","bedroom items",120)]

product_df = spark.createDataFrame(product,["product_id","pname","ptype","price"])
product_df.show()

+----------+-------+-------------+-----+
|product_id|  pname|        ptype|price|
+----------+-------+-------------+-----+
|         1|   cups|kitchen items|   50|
|         2| pencil|          art|   20|
|         3| paints|          art|   70|
|         4| spoons|kitchen items|   25|
|         5|pillows|bedroom items|  100|
|         6| mirror|bedroom items|  120|
+----------+-------+-------------+-----+



In [9]:
# Customer - cust_id, cname, mobileno, city

customer = [(10,"Jack","9876543213","Pune"),
           (11,"Ria","7865624351","Mumbai"),
           (12,"radha","9857654234","Pune"),
           (13,"janice","6788736472","Pune"),
           (14,"ray","78978967856","Mumbai"),
           (15,"adi","9876789098","Mumbai")]

customer_df = spark.createDataFrame(customer,["cust_id","cname","mobileno","city"])
customer_df.show()

+-------+------+-----------+------+
|cust_id| cname|   mobileno|  city|
+-------+------+-----------+------+
|     10|  Jack| 9876543213|  Pune|
|     11|   Ria| 7865624351|Mumbai|
|     12| radha| 9857654234|  Pune|
|     13|janice| 6788736472|  Pune|
|     14|   ray|78978967856|Mumbai|
|     15|   adi| 9876789098|Mumbai|
+-------+------+-----------+------+



In [10]:
# orders – order_id, order_date, order_customer_id, order_status

orders = [(100,"01-12-2024",11,"Pending"),
         (101,"12-11-2024",13,"Complete"),
         (102,"24-11-2024",14,"Shipped"),
         (103,"30-11-2024",10,"Pending"),
         (104,"23-11-2024",10,"Complete")]

orders_df = spark.createDataFrame(orders,["order_id","order_date","order_customer_id","order_status"])
orders_df.show()

+--------+----------+-----------------+------------+
|order_id|order_date|order_customer_id|order_status|
+--------+----------+-----------------+------------+
|     100|01-12-2024|               11|     Pending|
|     101|12-11-2024|               13|    Complete|
|     102|24-11-2024|               14|     Shipped|
|     103|30-11-2024|               10|     Pending|
|     104|23-11-2024|               10|    Complete|
+--------+----------+-----------------+------------+



In [11]:
# order_items –  order_item_order_id, order_item_product_id, order_item_quantity, order_item_subtotal

order_items = [(100,2,5,250),
              (101,4,5,125),
              (102,3,2,140),
              (103,4,3,75),
              (104,5,5,500)]

order_items_df = spark.createDataFrame(order_items,["order_item_order_id","order_item_product_id","order_item_quantity","order_item_subtotal"])

order_items_df.show()

+-------------------+---------------------+-------------------+-------------------+
|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|
+-------------------+---------------------+-------------------+-------------------+
|                100|                    2|                  5|                250|
|                101|                    4|                  5|                125|
|                102|                    3|                  2|                140|
|                103|                    4|                  3|                 75|
|                104|                    5|                  5|                500|
+-------------------+---------------------+-------------------+-------------------+



In [12]:
# Get details of all customers from pune city

customer_df.where(customer_df["city"]=="Pune").show()

+-------+------+----------+----+
|cust_id| cname|  mobileno|city|
+-------+------+----------+----+
|     10|  Jack|9876543213|Pune|
|     12| radha|9857654234|Pune|
|     13|janice|6788736472|Pune|
+-------+------+----------+----+



In [13]:
# Get details of orders with subtotal > ___ in month __

orders_join = orders_df.join(order_items_df,orders_df["order_id"]==order_items_df["order_item_order_id"])
orders_join.show()

+--------+----------+-----------------+------------+-------------------+---------------------+-------------------+-------------------+
|order_id|order_date|order_customer_id|order_status|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|
+--------+----------+-----------------+------------+-------------------+---------------------+-------------------+-------------------+
|     100|01-12-2024|               11|     Pending|                100|                    2|                  5|                250|
|     101|12-11-2024|               13|    Complete|                101|                    4|                  5|                125|
|     102|24-11-2024|               14|     Shipped|                102|                    3|                  2|                140|
|     103|30-11-2024|               10|     Pending|                103|                    4|                  3|                 75|
|     104|23-11-2024|               10|    Complete|   

In [14]:
customer_orders_join = customer_df.join(orders_join,customer_df["cust_id"] == orders_join["order_customer_id"])
customer_orders_join.show()

+-------+------+-----------+------+--------+----------+-----------------+------------+-------------------+---------------------+-------------------+-------------------+
|cust_id| cname|   mobileno|  city|order_id|order_date|order_customer_id|order_status|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|
+-------+------+-----------+------+--------+----------+-----------------+------------+-------------------+---------------------+-------------------+-------------------+
|     10|  Jack| 9876543213|  Pune|     104|23-11-2024|               10|    Complete|                104|                    5|                  5|                500|
|     10|  Jack| 9876543213|  Pune|     103|30-11-2024|               10|     Pending|                103|                    4|                  3|                 75|
|     11|   Ria| 7865624351|Mumbai|     100|01-12-2024|               11|     Pending|                100|                    2|                  5|       

In [16]:
cust_agg = customer_orders_join.groupBy("cust_id").agg(sum("order_item_subtotal"))
cust_agg.show()

+-------+------------------------+
|cust_id|sum(order_item_subtotal)|
+-------+------------------------+
|     10|                     575|
|     11|                     250|
|     13|                     125|
|     14|                     140|
+-------+------------------------+



In [18]:
customer_orders_join.groupBy("cust_id").agg(min("order_item_subtotal").alias("Min"), max("order_item_subtotal").alias("Max")).show()

+-------+---+---+
|cust_id|Min|Max|
+-------+---+---+
|     10| 75|500|
|     11|250|250|
|     13|125|125|
|     14|140|140|
+-------+---+---+



In [26]:
orders_join.filter(orders_join["order_date"].contains("11")).where(orders_join["order_item_subtotal"]>130).show()

+--------+----------+-----------------+------------+-------------------+---------------------+-------------------+-------------------+
|order_id|order_date|order_customer_id|order_status|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|
+--------+----------+-----------------+------------+-------------------+---------------------+-------------------+-------------------+
|     102|24-11-2024|               14|     Shipped|                102|                    3|                  2|                140|
|     104|23-11-2024|               10|    Complete|                104|                    5|                  5|                500|
+--------+----------+-----------------+------------+-------------------+---------------------+-------------------+-------------------+



In [27]:
# Print orders in ascending order of subtotal 

orders_join.orderBy("order_item_subtotal").show()

+--------+----------+-----------------+------------+-------------------+---------------------+-------------------+-------------------+
|order_id|order_date|order_customer_id|order_status|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|
+--------+----------+-----------------+------------+-------------------+---------------------+-------------------+-------------------+
|     103|30-11-2024|               10|     Pending|                103|                    4|                  3|                 75|
|     101|12-11-2024|               13|    Complete|                101|                    4|                  5|                125|
|     102|24-11-2024|               14|     Shipped|                102|                    3|                  2|                140|
|     100|01-12-2024|               11|     Pending|                100|                    2|                  5|                250|
|     104|23-11-2024|               10|    Complete|   

In [29]:
# in desending

orders_join.orderBy(desc("order_item_subtotal")).show()

+--------+----------+-----------------+------------+-------------------+---------------------+-------------------+-------------------+
|order_id|order_date|order_customer_id|order_status|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|
+--------+----------+-----------------+------------+-------------------+---------------------+-------------------+-------------------+
|     104|23-11-2024|               10|    Complete|                104|                    5|                  5|                500|
|     100|01-12-2024|               11|     Pending|                100|                    2|                  5|                250|
|     102|24-11-2024|               14|     Shipped|                102|                    3|                  2|                140|
|     101|12-11-2024|               13|    Complete|                101|                    4|                  5|                125|
|     103|30-11-2024|               10|     Pending|   

In [1]:
# Print customer details with min order and max order amount

customer_order = customer_df.join(orders_join, customer_df["cust_id"] == orders_join["order_customer_id"] )

cust_order_agg = customer_order.groupBy("order_customer_id").agg(sum("order_item_subtotal").alias("total_order_value"))

cust_order_agg.agg(min("total_order_value").alias("Lil"), max("total_order_value").alias("Big")).show()

NameError: name 'customer_df' is not defined

In [41]:
customer_details = customer_order.join(cust_order_agg, customer_order.cust_id == cust_order_agg.order_customer_id)
customer_details.show()

+-------+------+-----------+------+--------+----------+-----------------+------------+-------------------+---------------------+-------------------+-------------------+-----------------+-----------------+
|cust_id| cname|   mobileno|  city|order_id|order_date|order_customer_id|order_status|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|order_customer_id|total_order_value|
+-------+------+-----------+------+--------+----------+-----------------+------------+-------------------+---------------------+-------------------+-------------------+-----------------+-----------------+
|     10|  Jack| 9876543213|  Pune|     104|23-11-2024|               10|    Complete|                104|                    5|                  5|                500|               10|              575|
|     10|  Jack| 9876543213|  Pune|     103|30-11-2024|               10|     Pending|                103|                    4|                  3|                 75|            

In [45]:
# Get orders which are either COMPLETE or CLOSED and in dec 2024

orders_df.filter((orders_df.order_status == "Pending")|(orders_df.order_status == "Complete")).show()

+--------+----------+-----------------+------------+
|order_id|order_date|order_customer_id|order_status|
+--------+----------+-----------------+------------+
|     100|01-12-2024|               11|     Pending|
|     101|12-11-2024|               13|    Complete|
|     103|30-11-2024|               10|     Pending|
|     104|23-11-2024|               10|    Complete|
+--------+----------+-----------------+------------+



In [58]:
orders_df.filter(((orders_df.order_status == "Pending")|(orders_df.order_status == "Complete")) & (orders_df["order_date"].like("%%-12-2024"))).show()

+--------+----------+-----------------+------------+
|order_id|order_date|order_customer_id|order_status|
+--------+----------+-----------------+------------+
|     100|01-12-2024|               11|     Pending|
+--------+----------+-----------------+------------+



In [56]:
orders_df.filter((orders_df.order_status == "Pending")|(orders_df.order_status == "Complete").where(orders_df["order_date"].contains("%-12-2024"))).show()

TypeError: 'Column' object is not callable

In [19]:
# 2nd assignment

df = spark.read.csv("ecommerce_data.csv",header = "True")
df.show(5)

+---------+---------+--------------------+--------+----------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|     InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+----------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12-01-2010 08:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12-01-2010 08:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12-01-2010 08:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12-01-2010 08:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12-01-2010 08:26|     3.39|     17850|United Kingdom|
+---------+---------+--------------------+--------+----------------+---------+----------+--------------+
only showing top 5 rows



In [21]:
df = df.withColumn("Invoicedate",to_date(df["InvoiceDate"],"dd-MM-yyy HH:mm"))
df.show(5)

+---------+---------+--------------------+--------+-----------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|Invoicedate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-----------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6| 2010-01-12|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6| 2010-01-12|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8| 2010-01-12|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6| 2010-01-12|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6| 2010-01-12|     3.39|     17850|United Kingdom|
+---------+---------+--------------------+--------+-----------+---------+----------+--------------+
only showing top 5 rows



In [23]:
df.dropDuplicates(["InvoiceNo"]).show()

+---------+---------+--------------------+--------+-----------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|Invoicedate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-----------+---------+----------+--------------+
|   536366|    22633|HAND WARMER UNION...|       6| 2010-01-12|     1.85|     17850|United Kingdom|
|   536367|    84879|ASSORTED COLOUR B...|      32| 2010-01-12|     1.69|     13047|United Kingdom|
|   536371|    22086|PAPER CHAIN KIT 5...|      80| 2010-01-12|     2.55|     13748|United Kingdom|
|   536374|    21258|VICTORIAN SEWING ...|      32| 2010-01-12|    10.95|     15100|United Kingdom|
|   536375|   85123A|WHITE HANGING HEA...|       6| 2010-01-12|     2.55|     17850|United Kingdom|
|   536377|    22632|HAND WARMER RED P...|       6| 2010-01-12|     1.85|     17850|United Kingdom|
|   536384|    82484|WOOD BLACK BOARD ...|       3| 2010-01-12|     6.45|     18074|United Kingdom|


In [24]:
df.createOrReplaceTempView("temp_sale")
spark.sql("SELECT * from temp_sale WHERE Quantity > 150").show()

+---------+---------+--------------------+--------+-----------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|Invoicedate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-----------+---------+----------+--------------+
|   536387|    79321|       CHILLI LIGHTS|     192| 2010-01-12|     3.82|     16029|United Kingdom|
|   536387|    22780|LIGHT GARLAND BUT...|     192| 2010-01-12|     3.37|     16029|United Kingdom|
|   536387|    22779|WOODEN OWLS LIGHT...|     192| 2010-01-12|     3.37|     16029|United Kingdom|
|   536387|    22466|FAIRY TALE COTTAG...|     432| 2010-01-12|     1.45|     16029|United Kingdom|
|   536387|    21731|RED TOADSTOOL LED...|     432| 2010-01-12|     1.25|     16029|United Kingdom|
|   536390|    20668|DISCO BALL CHRIST...|     288| 2010-01-12|      0.1|     17511|United Kingdom|
|   536437|    21154|RED RETROSPOT OVE...|     200| 2010-01-12|     1.06|     13694|United Kingdom|


In [26]:
df.count()

541909

In [27]:
df.groupBy("Country").count().show()

+---------------+-----+
|        Country|count|
+---------------+-----+
|         Sweden|  462|
|      Singapore|  229|
|        Germany| 9495|
|         France| 8557|
|         Greece|  146|
|        Belgium| 2069|
|        Finland|  695|
|          Italy|  803|
|           EIRE| 8196|
|      Lithuania|   35|
|         Norway| 1086|
|          Spain| 2533|
|        Denmark|  389|
|      Hong Kong|  288|
|        Iceland|  182|
|         Israel|  297|
|Channel Islands|  758|
|         Cyprus|  622|
|    Switzerland| 2002|
|        Lebanon|   45|
+---------------+-----+
only showing top 20 rows



In [29]:
df.columns

['InvoiceNo',
 'StockCode',
 'Description',
 'Quantity',
 'Invoicedate',
 'UnitPrice',
 'CustomerID',
 'Country']

In [37]:
df = df.withColumn("Quantity", col("Quantity").cast("float"))

In [38]:
df = df.withColumn("UnitPrice", col("UnitPrice").cast("float"))

In [39]:
from pyspark.ml.feature import MinMaxScaler, VectorAssembler

assembler = VectorAssembler(inputCols = ["Quantity","UnitPrice"], outputCol = "vector")
df_vec = assembler.transform(df)
df_vec.show()

+---------+---------+--------------------+--------+-----------+---------+----------+--------------+--------------------+
|InvoiceNo|StockCode|         Description|Quantity|Invoicedate|UnitPrice|CustomerID|       Country|              vector|
+---------+---------+--------------------+--------+-----------+---------+----------+--------------+--------------------+
|   536365|   85123A|WHITE HANGING HEA...|     6.0| 2010-01-12|     2.55|     17850|United Kingdom|[6.0,2.5499999523...|
|   536365|    71053| WHITE METAL LANTERN|     6.0| 2010-01-12|     3.39|     17850|United Kingdom|[6.0,3.3900001049...|
|   536365|   84406B|CREAM CUPID HEART...|     8.0| 2010-01-12|     2.75|     17850|United Kingdom|          [8.0,2.75]|
|   536365|   84029G|KNITTED UNION FLA...|     6.0| 2010-01-12|     3.39|     17850|United Kingdom|[6.0,3.3900001049...|
|   536365|   84029E|RED WOOLLY HOTTIE...|     6.0| 2010-01-12|     3.39|     17850|United Kingdom|[6.0,3.3900001049...|
|   536365|    22752|SET 7 BABUS

In [40]:
scaler = MinMaxScaler(inputCol = "vector", outputCol = "ScaledVector")
df_scaled = scaler.fit(df_vec).transform(df_vec)

In [42]:
df_scaled.show()

+---------+---------+--------------------+--------+-----------+---------+----------+--------------+--------------------+--------------------+
|InvoiceNo|StockCode|         Description|Quantity|Invoicedate|UnitPrice|CustomerID|       Country|              vector|        ScaledVector|
+---------+---------+--------------------+--------+-----------+---------+----------+--------------+--------------------+--------------------+
|   536365|   85123A|WHITE HANGING HEA...|     6.0| 2010-01-12|     2.55|     17850|United Kingdom|[6.0,2.5499999523...|[0.50003703932341...|
|   536365|    71053| WHITE METAL LANTERN|     6.0| 2010-01-12|     3.39|     17850|United Kingdom|[6.0,3.3900001049...|[0.50003703932341...|
|   536365|   84406B|CREAM CUPID HEART...|     8.0| 2010-01-12|     2.75|     17850|United Kingdom|          [8.0,2.75]|[0.50004938576455...|
|   536365|   84029G|KNITTED UNION FLA...|     6.0| 2010-01-12|     3.39|     17850|United Kingdom|[6.0,3.3900001049...|[0.50003703932341...|
|   53

In [44]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder

In [47]:
index = StringIndexer(inputCol = "Country", outputCol = "Country_ind")
df_ind = index.fit(df).transform(df)
df_ind.select("Country_ind").show()

+-----------+
|Country_ind|
+-----------+
|        0.0|
|        0.0|
|        0.0|
|        0.0|
|        0.0|
|        0.0|
|        0.0|
|        0.0|
|        0.0|
|        0.0|
|        0.0|
|        0.0|
|        0.0|
|        0.0|
|        0.0|
|        0.0|
|        0.0|
|        0.0|
|        0.0|
|        0.0|
+-----------+
only showing top 20 rows



In [48]:
encoder = OneHotEncoder(inputCol = "Country_ind", outputCol = "Country_enc")
df_enc = encoder.fit(df_ind).transform(df_ind)
df_enc.select(["Country_ind","Country_enc"]).show()

+-----------+--------------+
|Country_ind|   COuntry_enc|
+-----------+--------------+
|        0.0|(37,[0],[1.0])|
|        0.0|(37,[0],[1.0])|
|        0.0|(37,[0],[1.0])|
|        0.0|(37,[0],[1.0])|
|        0.0|(37,[0],[1.0])|
|        0.0|(37,[0],[1.0])|
|        0.0|(37,[0],[1.0])|
|        0.0|(37,[0],[1.0])|
|        0.0|(37,[0],[1.0])|
|        0.0|(37,[0],[1.0])|
|        0.0|(37,[0],[1.0])|
|        0.0|(37,[0],[1.0])|
|        0.0|(37,[0],[1.0])|
|        0.0|(37,[0],[1.0])|
|        0.0|(37,[0],[1.0])|
|        0.0|(37,[0],[1.0])|
|        0.0|(37,[0],[1.0])|
|        0.0|(37,[0],[1.0])|
|        0.0|(37,[0],[1.0])|
|        0.0|(37,[0],[1.0])|
+-----------+--------------+
only showing top 20 rows



In [53]:
df_enc.select(["Country_ind","Country_enc"]).where(df_enc["Country_ind"]==37.0).show()

+-----------+-----------+
|Country_ind|Country_enc|
+-----------+-----------+
|       37.0| (37,[],[])|
|       37.0| (37,[],[])|
|       37.0| (37,[],[])|
|       37.0| (37,[],[])|
|       37.0| (37,[],[])|
|       37.0| (37,[],[])|
|       37.0| (37,[],[])|
|       37.0| (37,[],[])|
|       37.0| (37,[],[])|
|       37.0| (37,[],[])|
+-----------+-----------+



In [54]:
df = spark.read.csv(r"D:\Data Science\Pyspark\Housing.csv", header = "True")
df.show(5)

+--------+----+--------+---------+-------+--------+---------+--------+---------------+---------------+-------+--------+----------------+
|   price|area|bedrooms|bathrooms|stories|mainroad|guestroom|basement|hotwaterheating|airconditioning|parking|prefarea|furnishingstatus|
+--------+----+--------+---------+-------+--------+---------+--------+---------------+---------------+-------+--------+----------------+
|13300000|7420|       4|        2|      3|     yes|       no|      no|             no|            yes|      2|     yes|       furnished|
|12250000|8960|       4|        4|      4|     yes|       no|      no|             no|            yes|      3|      no|       furnished|
|12250000|9960|       3|        2|      2|     yes|       no|     yes|             no|             no|      2|     yes|  semi-furnished|
|12215000|7500|       4|        2|      2|     yes|       no|     yes|             no|            yes|      3|     yes|       furnished|
|11410000|7420|       4|        1|      2

In [56]:
# Group by 'Bedrooms' and calculate the average price

df.groupBy("bedrooms").agg(avg("price").alias("Avg_Price")).show()

+--------+------------------+
|bedrooms|         Avg_Price|
+--------+------------------+
|       3| 4954598.133333334|
|       5|         5819800.0|
|       6|         4791500.0|
|       1|         2712500.0|
|       4| 5729757.894736842|
|       2|3632022.0588235296|
+--------+------------------+



In [1]:
# car evaluation

import findspark
findspark.init()
import pyspark
sc = pyspark.SparkContext(appName = "Car_evaluation")
sc

<SparkContext master=local[*] appName=Car_evaluation>

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [3]:
spark = SparkSession.builder.getOrCreate()

In [4]:
df = spark.read.csv(r"D:\Data Science\car_evaluation.csv", header = "True")
df.show(5)

+------+------+---+---+-----+----+-----+
|vhigh0|vhigh1| 22| 23|small| low|unacc|
+------+------+---+---+-----+----+-----+
| vhigh| vhigh|  2|  2|small| med|unacc|
| vhigh| vhigh|  2|  2|small|high|unacc|
| vhigh| vhigh|  2|  2|  med| low|unacc|
| vhigh| vhigh|  2|  2|  med| med|unacc|
| vhigh| vhigh|  2|  2|  med|high|unacc|
+------+------+---+---+-----+----+-----+
only showing top 5 rows



In [6]:
df_named = df.select(
    df["vhigh0"].alias("buying_price"),
    df["vhigh1"].alias("maintainance_cost"),
    df["22"].alias("num_of_doors"),
    df["23"].alias("num_of_person"),
    df["small"].alias("lug_boot"),
    df["low"].alias("safety"),
    df["unacc"].alias("decision")
)

In [16]:
df_named.show(5)

+------------+-----------------+------------+-------------+--------+------+--------+
|buying_price|maintainance_cost|num_of_doors|num_of_person|lug_boot|safety|decision|
+------------+-----------------+------------+-------------+--------+------+--------+
|       vhigh|            vhigh|           2|            2|   small|   med|   unacc|
|       vhigh|            vhigh|           2|            2|   small|  high|   unacc|
|       vhigh|            vhigh|           2|            2|     med|   low|   unacc|
|       vhigh|            vhigh|           2|            2|     med|   med|   unacc|
|       vhigh|            vhigh|           2|            2|     med|  high|   unacc|
+------------+-----------------+------------+-------------+--------+------+--------+
only showing top 5 rows



In [30]:
df_ind = df_named

In [31]:
cols = ["num_of_doors","num_of_person","buying_price","maintainance_cost","lug_boot","safety","decision"]

In [32]:
from pyspark.ml.feature import StringIndexer

for col in cols:
    index = StringIndexer(inputCol = col, outputCol = col +"_ind")
    df_ind = index.fit(df_ind).transform(df_ind)

In [33]:
df_ind.show(5)

+------------+-----------------+------------+-------------+--------+------+--------+----------------+-----------------+----------------+---------------------+------------+----------+------------+
|buying_price|maintainance_cost|num_of_doors|num_of_person|lug_boot|safety|decision|num_of_doors_ind|num_of_person_ind|buying_price_ind|maintainance_cost_ind|lug_boot_ind|safety_ind|decision_ind|
+------------+-----------------+------------+-------------+--------+------+--------+----------------+-----------------+----------------+---------------------+------------+----------+------------+
|       vhigh|            vhigh|           2|            2|   small|   med|   unacc|             3.0|              2.0|             3.0|                  3.0|         2.0|       1.0|         0.0|
|       vhigh|            vhigh|           2|            2|   small|  high|   unacc|             3.0|              2.0|             3.0|                  3.0|         2.0|       0.0|         0.0|
|       vhigh|      

In [35]:
df_selected = df_ind.select("num_of_doors_ind","num_of_person_ind","buying_price_ind","maintainance_cost_ind","lug_boot_ind","safety_ind","decision_ind")
df_selected.show()

+----------------+-----------------+----------------+---------------------+------------+----------+------------+
|num_of_doors_ind|num_of_person_ind|buying_price_ind|maintainance_cost_ind|lug_boot_ind|safety_ind|decision_ind|
+----------------+-----------------+----------------+---------------------+------------+----------+------------+
|             3.0|              2.0|             3.0|                  3.0|         2.0|       1.0|         0.0|
|             3.0|              2.0|             3.0|                  3.0|         2.0|       0.0|         0.0|
|             3.0|              2.0|             3.0|                  3.0|         1.0|       2.0|         0.0|
|             3.0|              2.0|             3.0|                  3.0|         1.0|       1.0|         0.0|
|             3.0|              2.0|             3.0|                  3.0|         1.0|       0.0|         0.0|
|             3.0|              2.0|             3.0|                  3.0|         0.0|       2

In [36]:
# standardisation

from pyspark.ml.feature import VectorAssembler, StandardScaler

assembler = VectorAssembler(inputCols = df_selected.columns, outputCol = "vector")
df_vec = assembler.transform(df_selected)

In [37]:
df_vec.show(5)

+----------------+-----------------+----------------+---------------------+------------+----------+------------+--------------------+
|num_of_doors_ind|num_of_person_ind|buying_price_ind|maintainance_cost_ind|lug_boot_ind|safety_ind|decision_ind|              vector|
+----------------+-----------------+----------------+---------------------+------------+----------+------------+--------------------+
|             3.0|              2.0|             3.0|                  3.0|         2.0|       1.0|         0.0|[3.0,2.0,3.0,3.0,...|
|             3.0|              2.0|             3.0|                  3.0|         2.0|       0.0|         0.0|[3.0,2.0,3.0,3.0,...|
|             3.0|              2.0|             3.0|                  3.0|         1.0|       2.0|         0.0|[3.0,2.0,3.0,3.0,...|
|             3.0|              2.0|             3.0|                  3.0|         1.0|       1.0|         0.0|[3.0,2.0,3.0,3.0,...|
|             3.0|              2.0|             3.0|         

In [38]:
scaler = StandardScaler(inputCol = "vector", outputCol = "scaled_vector")
df_scaled = scaler.fit(df_vec).transform(df_vec)

In [39]:
df_scaled.show(5)

+----------------+-----------------+----------------+---------------------+------------+----------+------------+--------------------+--------------------+
|num_of_doors_ind|num_of_person_ind|buying_price_ind|maintainance_cost_ind|lug_boot_ind|safety_ind|decision_ind|              vector|       scaled_vector|
+----------------+-----------------+----------------+---------------------+------------+----------+------------+--------------------+--------------------+
|             3.0|              2.0|             3.0|                  3.0|         2.0|       1.0|         0.0|[3.0,2.0,3.0,3.0,...|[2.68312693410585...|
|             3.0|              2.0|             3.0|                  3.0|         2.0|       0.0|         0.0|[3.0,2.0,3.0,3.0,...|[2.68312693410585...|
|             3.0|              2.0|             3.0|                  3.0|         1.0|       2.0|         0.0|[3.0,2.0,3.0,3.0,...|[2.68312693410585...|
|             3.0|              2.0|             3.0|                 

In [ ]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol = "scaled_vector", labelCol = "decision_ind")
predictions = lr.fit(df_scaled).transform(df_scaled)

In [46]:
predictions.select("scaled_vector","decision_ind","prediction").show()

+--------------------+------------+----------+
|       scaled_vector|decision_ind|prediction|
+--------------------+------------+----------+
|[2.68312693410585...|         0.0|       0.0|
|[2.68312693410585...|         0.0|       0.0|
|[2.68312693410585...|         0.0|       0.0|
|[2.68312693410585...|         0.0|       0.0|
|[2.68312693410585...|         0.0|       0.0|
|[2.68312693410585...|         0.0|       0.0|
|[2.68312693410585...|         0.0|       0.0|
|[2.68312693410585...|         0.0|       0.0|
|[2.68312693410585...|         0.0|       0.0|
|[2.68312693410585...|         0.0|       0.0|
|[2.68312693410585...|         0.0|       0.0|
|[2.68312693410585...|         0.0|       0.0|
|[2.68312693410585...|         0.0|       0.0|
|[2.68312693410585...|         0.0|       0.0|
|[2.68312693410585...|         0.0|       0.0|
|[2.68312693410585...|         0.0|       0.0|
|(7,[0,2,3],[2.683...|         0.0|       0.0|
|[2.68312693410585...|         0.0|       0.0|
|[2.683126934

In [51]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(labelCol = "decision_ind", rawPredictionCol = "prediction")

accuracy = evaluator.evaluate(predictions)

print(accuracy)

1.0


In [49]:
# Calculate confusion matrix
from pyspark.sql.functions import col

# Create a confusion matrix
confusion_matrix = predictions.groupBy("decision_ind", "prediction").count().show()

+------------+----------+-----+
|decision_ind|prediction|count|
+------------+----------+-----+
|         1.0|       1.0|  384|
|         2.0|       2.0|   69|
|         0.0|       0.0| 1209|
|         3.0|       3.0|   65|
+------------+----------+-----+



In [56]:
doc = r"D:\Data Science\abc.txt"

lines = sc.textFile(doc)

In [57]:
words = lines.flatMap(lambda line: line.split(" ")).filter(lambda word: word.strip() != "")

In [58]:
total = words.count()

In [59]:
total

29

In [61]:
unique_words = words.distinct().collect()

In [62]:
unique_words

['Hello,',
 'this',
 'assignment',
 'Subject',
 'Big',
 'new',
 'line',
 'added',
 'to',
 'text',
 'file.',
 'is',
 '5.',
 ':',
 'data',
 'This',
 'a',
 'the']

## Student admission

In [1]:
# Student attendance - ml model

import findspark
findspark.init()
import pyspark

In [2]:
sc = pyspark.SparkContext(appName = "ml model")
sc

<SparkContext master=local[*] appName=ml model>

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [4]:
spark = SparkSession.builder.getOrCreate()

In [30]:
df = spark.read.csv(r"D:\Data Science\student_admission_record_dirty.csv", header = "True")
df.show(5)

+-------+----+------+--------------------+----------------------+---------+----------------+
|   Name| Age|Gender|Admission Test Score|High School Percentage|     City|Admission Status|
+-------+----+------+--------------------+----------------------+---------+----------------+
|Shehroz|24.0|Female|                50.0|                  68.9|   Quetta|        Rejected|
|  Waqar|21.0|Female|                99.0|                 60.73|  Karachi|            NULL|
| Bushra|17.0|  Male|                89.0|                  NULL|Islamabad|        Accepted|
|  Aliya|17.0|  Male|                55.0|                 85.29|  Karachi|        Rejected|
|  Bilal|20.0|  Male|                65.0|                 61.13|   Lahore|            NULL|
+-------+----+------+--------------------+----------------------+---------+----------------+
only showing top 5 rows



In [31]:
df = df.dropna()

In [32]:
df.show()

+-------+----+------+--------------------+----------------------+----------+----------------+
|   Name| Age|Gender|Admission Test Score|High School Percentage|      City|Admission Status|
+-------+----+------+--------------------+----------------------+----------+----------------+
|Shehroz|24.0|Female|                50.0|                  68.9|    Quetta|        Rejected|
|  Aliya|17.0|  Male|                55.0|                 85.29|   Karachi|        Rejected|
|  Rabia|20.0|Female|                82.0|                 55.67|    Lahore|        Accepted|
| Kamran|18.0|  Male|                53.0|                 98.98|    Multan|        Rejected|
| Shafiq|17.0|  Male|                78.0|                   -10|    Quetta|        Rejected|
|  Ahmed|21.0|  Male|                62.0|                 79.03|   Karachi|        Accepted|
|Fareeha|22.0|  Male|                86.0|                 50.77| Islamabad|        Rejected|
|   Saim|19.0|  Male|                84.0|                 9

In [33]:
df = df.filter(df["High School Percentage"]>50.00)
df.show()

+-------+----+------+--------------------+----------------------+----------+----------------+
|   Name| Age|Gender|Admission Test Score|High School Percentage|      City|Admission Status|
+-------+----+------+--------------------+----------------------+----------+----------------+
|Shehroz|24.0|Female|                50.0|                  68.9|    Quetta|        Rejected|
|  Aliya|17.0|  Male|                55.0|                 85.29|   Karachi|        Rejected|
|  Rabia|20.0|Female|                82.0|                 55.67|    Lahore|        Accepted|
| Kamran|18.0|  Male|                53.0|                 98.98|    Multan|        Rejected|
|  Ahmed|21.0|  Male|                62.0|                 79.03|   Karachi|        Accepted|
|Fareeha|22.0|  Male|                86.0|                 50.77| Islamabad|        Rejected|
|   Saim|19.0|  Male|                84.0|                 97.19|  Peshawar|        Rejected|
|   Uzma|17.0|Female|                96.0|                 9

In [34]:
df.count()

98

In [35]:
df = df.filter(df["Admission Test Score"]>50.00)
df.show()

+-------+----+------+--------------------+----------------------+----------+----------------+
|   Name| Age|Gender|Admission Test Score|High School Percentage|      City|Admission Status|
+-------+----+------+--------------------+----------------------+----------+----------------+
|  Aliya|17.0|  Male|                55.0|                 85.29|   Karachi|        Rejected|
|  Rabia|20.0|Female|                82.0|                 55.67|    Lahore|        Accepted|
| Kamran|18.0|  Male|                53.0|                 98.98|    Multan|        Rejected|
|  Ahmed|21.0|  Male|                62.0|                 79.03|   Karachi|        Accepted|
|Fareeha|22.0|  Male|                86.0|                 50.77| Islamabad|        Rejected|
|   Saim|19.0|  Male|                84.0|                 97.19|  Peshawar|        Rejected|
|   Uzma|17.0|Female|                96.0|                 93.34|   Karachi|        Rejected|
| Maryam|19.0|Female|                74.0|                 7

In [36]:
df.count()

94

In [37]:
df.groupBy("City").count().show()

+----------+-----+
|      City|count|
+----------+-----+
|    Multan|   19|
|   Karachi|   17|
|  Peshawar|   15|
|    Lahore|   11|
|Rawalpindi|    7|
| Islamabad|    8|
|    Quetta|   17|
+----------+-----+



In [38]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, MinMaxScaler, VectorAssembler

df_ind = df
cat_cols = ["Age","Admission Test Score","High School Percentage","Gender", "City", "Admission Status"]

for col in cat_cols:
    indexer = StringIndexer(inputCol = col, outputCol = col + "_ind")
    df_ind = indexer.fit(df_ind).transform(df_ind)



In [39]:
cat_cols_to_encode = ["Age_ind","Admission Test Score_ind","High School Percentage_ind","Gender_ind", "City_ind"]

df_enc = df_ind
for col in cat_cols_to_encode:
    encoder = OneHotEncoder(inputCol = col, outputCol = col + "_enc")
    df_enc = encoder.fit(df_enc).transform(df_enc)

In [40]:
df_enc.show(3)

+------+----+------+--------------------+----------------------+-------+----------------+-------+------------------------+--------------------------+----------+--------+--------------------+-------------+----------------------------+------------------------------+--------------+-------------+
|  Name| Age|Gender|Admission Test Score|High School Percentage|   City|Admission Status|Age_ind|Admission Test Score_ind|High School Percentage_ind|Gender_ind|City_ind|Admission Status_ind|  Age_ind_enc|Admission Test Score_ind_enc|High School Percentage_ind_enc|Gender_ind_enc| City_ind_enc|
+------+----+------+--------------------+----------------------+-------+----------------+-------+------------------------+--------------------------+----------+--------+--------------------+-------------+----------------------------+------------------------------+--------------+-------------+
| Aliya|17.0|  Male|                55.0|                 85.29|Karachi|        Rejected|    0.0|                    3

In [41]:
df_enc = df_enc.select("Age_ind_enc","Admission Test Score_ind_enc","High School Percentage_ind_enc","Gender_ind_enc","City_ind_enc","Admission Status_ind")

In [42]:
df_enc.show(5)

+-------------+----------------------------+------------------------------+--------------+-------------+--------------------+
|  Age_ind_enc|Admission Test Score_ind_enc|High School Percentage_ind_enc|Gender_ind_enc| City_ind_enc|Admission Status_ind|
+-------------+----------------------------+------------------------------+--------------+-------------+--------------------+
|(8,[0],[1.0])|             (40,[30],[1.0])|                (86,[2],[1.0])|     (1,[],[])|(6,[1],[1.0])|                 0.0|
|(8,[1],[1.0])|              (40,[6],[1.0])|               (86,[16],[1.0])| (1,[0],[1.0])|(6,[4],[1.0])|                 1.0|
|(8,[7],[1.0])|             (40,[16],[1.0])|                (86,[6],[1.0])|     (1,[],[])|(6,[0],[1.0])|                 0.0|
|(8,[6],[1.0])|             (40,[18],[1.0])|                (86,[1],[1.0])|     (1,[],[])|(6,[1],[1.0])|                 1.0|
|(8,[2],[1.0])|                  (40,[],[])|                (86,[8],[1.0])|     (1,[],[])|(6,[5],[1.0])|              

In [43]:
vector = VectorAssembler(inputCols = ["Age_ind_enc","Admission Test Score_ind_enc","High School Percentage_ind_enc","Gender_ind_enc","City_ind_enc"], outputCol = "vector")
df_vec = vector.transform(df_enc)

In [44]:
scaler = MinMaxScaler(inputCol = "vector", outputCol = "vector_scaled")
df_vec = scaler.fit(df_vec).transform(df_vec)

In [45]:
df_vec.select("vector","vector_scaled").show()

+--------------------+--------------------+
|              vector|       vector_scaled|
+--------------------+--------------------+
|(141,[0,38,50,136...|(141,[0,38,50,136...|
|(141,[1,14,64,134...|(141,[1,14,64,134...|
|(141,[7,24,54,135...|(141,[7,24,54,135...|
|(141,[6,26,49,136...|(141,[6,26,49,136...|
|(141,[2,56,140],[...|(141,[2,56,140],[...|
|(141,[4,17,130,13...|(141,[4,17,130,13...|
|(141,[0,19,125,13...|(141,[0,19,125,13...|
|(141,[4,9,88,134,...|(141,[4,9,88,134,...|
|(141,[2,12,126,13...|(141,[2,12,126,13...|
|(141,[7,24,54,135...|(141,[7,24,54,135...|
|(141,[1,45,86,134...|(141,[1,45,86,134...|
|(141,[1,19,71,134...|(141,[1,19,71,134...|
|(141,[0,18,101,14...|(141,[0,18,101,14...|
|(141,[0,9,65],[1....|(141,[0,9,65],[1....|
|(141,[3,27,121,13...|(141,[3,27,121,13...|
|(141,[3,42,95,134...|(141,[3,42,95,134...|
|(141,[3,25,113,13...|(141,[3,25,113,13...|
|(141,[3,15,53,135...|(141,[3,15,53,135...|
|(141,[6,18,107,13...|(141,[6,18,107,13...|
|(141,[6,41,96,140...|(141,[6,41

In [46]:
from pyspark.ml.classification import DecisionTreeClassifier

dtc = DecisionTreeClassifier(featuresCol = "vector_scaled", labelCol = "Admission Status_ind")

predictions = dtc.fit(df_vec).transform(df_vec)

In [54]:
from pyspark.ml.classification import RandomForestClassifier

dtc = RandomForestClassifier(featuresCol = "vector_scaled", labelCol = "Admission Status_ind")

predictions = dtc.fit(df_vec).transform(df_vec)

In [55]:
predictions.select("vector_scaled","Admission Status_ind","prediction").show()

+--------------------+--------------------+----------+
|       vector_scaled|Admission Status_ind|prediction|
+--------------------+--------------------+----------+
|(141,[0,38,50,136...|                 0.0|       0.0|
|(141,[1,14,64,134...|                 1.0|       1.0|
|(141,[7,24,54,135...|                 0.0|       0.0|
|(141,[6,26,49,136...|                 1.0|       1.0|
|(141,[2,56,140],[...|                 0.0|       0.0|
|(141,[4,17,130,13...|                 0.0|       0.0|
|(141,[0,19,125,13...|                 0.0|       0.0|
|(141,[4,9,88,134,...|                 1.0|       1.0|
|(141,[2,12,126,13...|                 0.0|       0.0|
|(141,[7,24,54,135...|                 0.0|       0.0|
|(141,[1,45,86,134...|                 0.0|       0.0|
|(141,[1,19,71,134...|                 0.0|       0.0|
|(141,[0,18,101,14...|                 1.0|       0.0|
|(141,[0,9,65],[1....|                 0.0|       0.0|
|(141,[3,27,121,13...|                 0.0|       0.0|
|(141,[3,4

In [56]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol = "Admission Status_ind", predictionCol = "prediction", metricName = "accuracy")
print(evaluator.evaluate(predictions))

0.7659574468085106


In [57]:
evaluator = MulticlassClassificationEvaluator(labelCol = "Admission Status_ind", predictionCol = "prediction", metricName = "weightedPrecision")
print(evaluator.evaluate(predictions))

0.8222016651248844


In [58]:
evaluator = MulticlassClassificationEvaluator(labelCol = "Admission Status_ind", predictionCol = "prediction", metricName = "weightedRecall")
print(evaluator.evaluate(predictions))

0.7659574468085106


In [59]:
evaluator = MulticlassClassificationEvaluator(labelCol = "Admission Status_ind", predictionCol = "prediction", metricName = "f1")
print(evaluator.evaluate(predictions))

0.7523569110298285


## File handling

In [60]:
# File handling

doc = r"D:\Data Science\abc.txt"

In [61]:
df = spark.read.text(doc)

print("Original data: ")
df.show(truncate = False)

Original data: 
+------------------------------------------+
|value                                     |
+------------------------------------------+
|Hello, this is assignment 5.              |
|Subject : Big data                        |
|This is a new line added to the text file.|
|This is a new line added to the text file.|
+------------------------------------------+



In [64]:
from pyspark.sql import Row

new_line = '''NEW LINE 1.
NEW LINE 2'''
df_new = spark.createDataFrame([Row(value = new_line)])

df = df.union(df_new)

In [65]:
df.show(truncate = False)

+------------------------------------------+
|value                                     |
+------------------------------------------+
|Hello, this is assignment 5.              |
|Subject : Big data                        |
|This is a new line added to the text file.|
|This is a new line added to the text file.|
|NEW LINE 1.\nNEW LINE 2                   |
+------------------------------------------+



In [66]:
updated_line = df.select("value").rdd.flatMap(lambda x : x).collect()

In [67]:
with open(doc,"w") as f:
    for line in updated_line:
        f.write(line + "\n")

In [70]:
# count the number of words and unique words print

lines = sc.textFile(doc)

In [72]:
words = lines.flatMap(lambda line: line.split(" ")).filter(lambda word : word.strip != (""))

In [73]:
words.count()

35

In [74]:
words.distinct().collect()

['Hello,',
 'this',
 'assignment',
 'Subject',
 'Big',
 'new',
 'line',
 'added',
 'to',
 'text',
 'file.',
 '1.',
 'is',
 '5.',
 ':',
 'data',
 'This',
 'a',
 'the',
 'NEW',
 'LINE',
 '2']

In [76]:
df.show(truncate=False)

+------------------------------------------+
|value                                     |
+------------------------------------------+
|Hello, this is assignment 5.              |
|Subject : Big data                        |
|This is a new line added to the text file.|
|This is a new line added to the text file.|
|NEW LINE 1.                               |
|NEW LINE 1.\nNEW LINE 2                   |
+------------------------------------------+



In [82]:
student = [(100,"Anushree",22),(101,"Sonali",23),(102,"Ekta",23)]
teacher = [(10,"Smita","ML"),(11,"Uma","Placement"),(12,"Rasika","DL")]
subject = [(1,"Python",10),(2,"Maths",10),(3,"Stats",11),(4,"ML",12),(5,"DL",12)]
student_sub = [(100,1),(100,3),(100,4),(101,1),(101,4),(102,5),(100,5)]

student_df = spark.createDataFrame(student,["rno","sname","age"])

In [83]:
student_df.show()

+---+--------+---+
|rno|   sname|age|
+---+--------+---+
|100|Anushree| 22|
|101|  Sonali| 23|
|102|    Ekta| 23|
+---+--------+---+



In [86]:
student_df = student_df.selectExpr("rno","sname","age + 5")

In [87]:
student_df.show()

+---+--------+---------+
|rno|   sname|(age + 5)|
+---+--------+---------+
|100|Anushree|       27|
|101|  Sonali|       28|
|102|    Ekta|       28|
+---+--------+---------+



## New

In [2]:
import findspark
findspark.init()
import pyspark
sc = pyspark.SparkContext(appName = "new")
sc

<SparkContext master=local[*] appName=new>

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.getOrCreate()

In [5]:
student = [(100,"Anushree",22),(101,"Sonali",23),(102,"Ekta",23),(103,"Riyansha",24),(104,"Pratik",21)]
student_df = spark.createDataFrame(student,["rno","name","age"])

In [6]:
student_df.show()

+---+--------+---+
|rno|    name|age|
+---+--------+---+
|100|Anushree| 22|
|101|  Sonali| 23|
|102|    Ekta| 23|
|103|Riyansha| 24|
|104|  Pratik| 21|
+---+--------+---+



In [7]:
student_df.count()

5

In [8]:
student_df.columns

['rno', 'name', 'age']

In [9]:
student_df.dtypes

[('rno', 'bigint'), ('name', 'string'), ('age', 'bigint')]

In [10]:
student_df.printSchema()

root
 |-- rno: long (nullable = true)
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)



In [11]:
student_df.select(["rno","age"]).show()

+---+---+
|rno|age|
+---+---+
|100| 22|
|101| 23|
|102| 23|
|103| 24|
|104| 21|
+---+---+



In [12]:
student_df.filter(student_df.age >23).show()

+---+--------+---+
|rno|    name|age|
+---+--------+---+
|103|Riyansha| 24|
+---+--------+---+



In [14]:
student_df.groupBy("age").count().show()

+---+-----+
|age|count|
+---+-----+
| 22|    1|
| 23|    2|
| 24|    1|
| 21|    1|
+---+-----+



In [17]:
student_df.agg(avg("age")).show()

+--------+
|avg(age)|
+--------+
|    22.6|
+--------+



In [18]:
student_df.agg(avg("age").alias("avg_age"), sum("age").alias("sum_age")).show()

+-------+-------+
|avg_age|sum_age|
+-------+-------+
|   22.6|    113|
+-------+-------+



In [20]:
student_df.selectExpr("age","age + 5 as new_age").show()

+---+-------+
|age|new_age|
+---+-------+
| 22|     27|
| 23|     28|
| 23|     28|
| 24|     29|
| 21|     26|
+---+-------+



In [21]:
student_df.withColumnRenamed("sname","name").show()

+---+--------+---+
|rno|    name|age|
+---+--------+---+
|100|Anushree| 22|
|101|  Sonali| 23|
|102|    Ekta| 23|
|103|Riyansha| 24|
|104|  Pratik| 21|
+---+--------+---+



In [22]:
student_df.filter(student_df["age"].like("%3")).show()

+---+------+---+
|rno|  name|age|
+---+------+---+
|101|Sonali| 23|
|102|  Ekta| 23|
+---+------+---+



In [26]:
student_df.withColumn("age",col("age").cast("float")).show()

+---+--------+----+
|rno|    name| age|
+---+--------+----+
|100|Anushree|22.0|
|101|  Sonali|23.0|
|102|    Ekta|23.0|
|103|Riyansha|24.0|
|104|  Pratik|21.0|
+---+--------+----+



In [28]:
student_df.withColumn("City",lit("Pune")).show()

+---+--------+---+----+
|rno|    name|age|City|
+---+--------+---+----+
|100|Anushree| 22|Pune|
|101|  Sonali| 23|Pune|
|102|    Ekta| 23|Pune|
|103|Riyansha| 24|Pune|
|104|  Pratik| 21|Pune|
+---+--------+---+----+



In [29]:
student_df.createOrReplaceTempView("student_view")

In [30]:
spark.sql("select * from student_view").show()

+---+--------+---+
|rno|    name|age|
+---+--------+---+
|100|Anushree| 22|
|101|  Sonali| 23|
|102|    Ekta| 23|
|103|Riyansha| 24|
|104|  Pratik| 21|
+---+--------+---+



In [57]:
file = r"D:\Data Science\abc.txt"
df = spark.read.text(file)

In [58]:
# print
df.show(truncate = False)

+-------------------+
|value              |
+-------------------+
|Hello all          |
|this is a text file|
|deva so many issues|
+-------------------+



In [59]:
from pyspark.sql import Row
new_line = "another new line 1"

In [60]:
df_new = spark.createDataFrame([Row(value = new_line)])

In [61]:
df= df.union(df_new)

In [62]:
updated_lines = df.select("value").rdd.flatMap(lambda x : x).collect()

In [63]:
with open(file,"w") as f:
    for line in updated_lines:
        f.write(line + "\n")

In [64]:
df.show(truncate = False)

+-------------------+
|value              |
+-------------------+
|Hello all          |
|this is a text file|
|deva so many issues|
|another new line 1 |
|another new line 1 |
+-------------------+



In [66]:
lines = sc.textFile(file)
words = lines.flatMap(lambda line : line.split(" ")).filter(lambda word : word.strip() != "")

In [68]:
words.count()

15

In [69]:
words.distinct().collect()

['this',
 'text',
 'file',
 'deva',
 'so',
 'issues',
 'another',
 'new',
 'line',
 'Hello',
 'all',
 'is',
 'a',
 'many',
 '1']

In [71]:
student_df.sort("name").show()

+---+--------+---+
|rno|    name|age|
+---+--------+---+
|100|Anushree| 22|
|102|    Ekta| 23|
|104|  Pratik| 21|
|103|Riyansha| 24|
|101|  Sonali| 23|
+---+--------+---+

